# [Lexicographic permutations](https://projecteuler.net/problem=24)

<blockquote>
<p>A permutation is an ordered arrangement of objects. For example, 3124 is one possible permutation of the digits 1, 2, 3 and 4. If all of the permutations are listed numerically or alphabetically, we call it lexicographic order. The lexicographic permutations of 0, 1 and 2 are:</p>
<p style="text-align:center;">012   021   102   120   201   210</p>
<p>What is the millionth lexicographic permutation of the digits 0, 1, 2, 3, 4, 5, 6, 7, 8 and 9?</p>
</blockquote>

Let’s start by trying to reproduce the example:

In [1]:
D = '0':'2'
for a ∈ D, b ∈ D, c ∈ D
    P = [a, b, c]
    allunique(P) && (println ∘ join)(P)
end

012
021
102
120
201
210


Loop through all the digits in order. Print them out when each appears just once. Easy.

Can I just adapt this to ten digits instead of three? That *would* be $10^{10}$ iterations, but maybe the compiler will be able to optimize it to run in a reasonable amount of time.

In [2]:
D = '0':'9'
n = 0
for a ∈ D, b ∈ D, c ∈ D, d ∈ D, e ∈ D, f ∈ D, g ∈ D, h ∈ D, i ∈ D, j ∈ D
    P = [a, b, c, d, e, f, g, h, i, j]
    if allunique(P)
        n += 1
        if n == 1_000_000
            (println ∘ join)(P)
            break
        end
    end
end

InterruptException: InterruptException:

Hmmm… nothing happening, fans spinning up, still nothing happening… OK, time to abort and try another approach.

Let’s increase the number of digits by one and see if a pattern emerges.

In [3]:
D = '0':'3'
for a ∈ D, b ∈ D, c ∈ D, d ∈ D
    P = [a, b, c, d]
    allunique(P) && (println ∘ join)(P)
end

0123
0132
0213
0231
0312
0321
1023
1032
1203
1230
1302
1320
2013
2031
2103
2130
2301
2310
3012
3021
3102
3120
3201
3210


In [4]:
D = '0':'4'
for a ∈ D, b ∈ D, c ∈ D, d ∈ D, e ∈ D
    P = [a, b, c, d, e]
    allunique(P) && (println ∘ join)(P)
end

01234
01243
01324
01342
01423
01432
02134
02143
02314
02341
02413
02431
03124
03142
03214
03241
03412
03421
04123
04132
04213
04231
04312
04321
10234
10243
10324
10342
10423
10432
12034
12043
12304
12340
12403
12430
13024
13042
13204
13240
13402
13420
14023
14032
14203
14230
14302
14320
20134
20143
20314
20341
20413
20431
21034
21043
21304
21340
21403
21430
23014
23041
23104
23140
23401
23410
24013
24031
24103
24130
24301
24310
30124
30142
30214
30241
30412
30421
31024
31042
31204
31240
31402
31420
32014
32041
32104
32140
32401
32410
34012
34021
34102
34120
34201
34210
40123
40132
40213
40231
40312
40321
41023
41032
41203
41230
41302
41320
42013
42031
42103
42130
42301
42310
43012
43021
43102
43120
43201
43210


OK, the 5th digit appears in runs of 24, the 4th in runs of 6, the 3rd in 2, and the 2nd and 1st in 1. In increasing order: 1, 1, 2, 6, 24, …. That sounds familiar. Let’s see what [OEIS has to say about that](https://oeis.org/search?q=1%2C1%2C2%2C6%2C24):

<blockquote>
<table>
<tbody>
<tr>
  <td><a href="https://oeis.org/A000142">A000142</a></td>
  <td>Factorial numbers: <i>n</i>! = 1×2×3×4× … ×<i>n</i> (order of symmetric group <i>S<sub>n</sub></i>, number of permutations of <i>n</i> letters).</td>
</tr>
<tr>
<td colspan="2"><b>1, 1, 2, 6, 24</b>, 120, 720, 5040, 40320, 362880,
3628800, 39916800, 479001600, 6227020800, 87178291200, 1307674368000, 20922789888000, 355687428096000,
6402373705728000, 121645100408832000, 2432902008176640000, 51090942171709440000, 1124000727777607680000
<span style="font-size: smaller">(
  <a href="https://oeis.org/A000142/list">list</a>;
  <a href="https://oeis.org/A000142/graph">graph</a>;
  <a href="https://oeis.org/search?q=A000142+-id:A000142">refs</a>;
  <a href="https://oeis.org/A000142/listen">listen</a>;
  <a href="https://oeis.org/history?seq=A000142">history</a>;
  <a href="https://oeis.org/search?q=id:A000142&amp;fmt=text">text</a>;
  <a href="https://oeis.org/A000142/internal">internal format</a>
  )</span>
</td>
</tr>
</tbody>
</table>
</blockquote>

Of course! Factorials make sense: $n$ choices for the first digit, $n-1$ for the next, etc.

So, for the left-most digit, they repeat in runs of $(m-1)!$ where $m$ is the number of digits. If we’re looking for the $n$th permutation that digit should be at position $n \div (m-1)!$ (*integer division, zero-based indexing*) in the list of available digits. The we need to remove that digit from the list and look for the $n \% (m-1)!$th permutation in that list. 

In [5]:
function pick(n, S)
    s = length(S) - 1
    s > 0 || return S[1]
    f = factorial(s)
    a, b = divrem(n, f)
    d = S[1+a]
    string(d, pick(b, setdiff(S, [d])))
end

pick(999999, collect('0':'9')) # 2783915460

"2783915460"

Great! Although, Julia’s penchant for 1-based indexing isn’t doing me any favors in this case… I wonder if there is a more Julia-idiomatic way of doing this…

In [6]:
using Combinatorics
join(collect(permutations('0':'9'))[1_000_000])

"2783915460"

🙄